In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score


from tensorflow.keras import layers, models, callbacks
from sklearn.pipeline import Pipeline

2025-11-25 00:24:13.664929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764030254.041452      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764030254.161399      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow: ", tf.__version__)

TensorFlow:  2.18.0


In [3]:
URL = '/kaggle/input/titanic/train_and_test2.csv'

df = pd.read_csv(URL)
print("Shape bruto: ", df.shape)
print("Cols: ", list(df.columns))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/titanic/train_and_test2.csv'

In [ ]:
y = df["2urvived"].astype(int).values

#Tomamos columnas usuales y evitamos leakage (PassengerId, name, Ticket, CABIN NO ayudan de base)
X = df[[
    "Age", "Fare", "Sex", "sibsp", "Embarked", "Pclass"
]].copy()


In [ ]:
num_cols = ["Age", "sibsp", "Fare", "Sex", "Embarked", "Pclass"] 

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()) # denso; aqui si con media
])


In [ ]:
from sklearn.pipeline import Pipeline

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols)
    ],
    remainder="drop"

)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_df, X_test_df, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

In [ ]:
X_train = preprocess.fit_transform(X_train_df)
X_test = preprocess.transform(X_test_df)

In [ ]:

X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [ ]:
def build_model(input_dim: int) -> tf.keras.Model:
  model = models.Sequential([
      layers.Input(shape=(input_dim,)),
      layers.Dense(32, activation="relu"),
      layers.Dropout(0.15), 
      layers.Dense(16, activation="relu"),
      layers.Dropout(0.15),
      layers.Dense(1, activation="sigmoid")
  ])
  model.compile(
      optimizer="adam",
      loss="binary_crossentropy", #perdida binaria
      metrics=["accuracy", tf.keras.metrics.AUC(name="auc")] #accuracy del modelo
  )
  return model

model = build_model(X_train.shape[1])
model.summary()

In [ ]:

cbs = [
    callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=12, restore_best_weights=True),
    callbacks.ModelCheckpoint(filepath="titanic_best.keras", monitor="val_auc", mode="max", save_best_only=True), #Se realiza un checkpoint o guardado para el callback
    callbacks.ReduceLROnPlateau(monitor="val_auc", factor=0.5, patience=6)
]

In [ ]:
hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=200,
    callbacks=cbs,
    verbose=1
)


In [ ]:
y_proba = model.predict(X_test).ravel()
y_pred = (y_proba >= 0.5).astype(int)

print("\nMatriz de confusion:\n", confusion_matrix(y_test, y_pred))
print("\nReporte de clasificacion:\n", classification_report(y_test, y_pred, digits=4))
print("\nAUC: ", roc_auc_score(y_test, y_proba))

In [ ]:
# REGRESIÓN LOGÍSTICA

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, roc_curve, auc
)

from matplotlib.colors import ListedColormap

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

In [ ]:
print("--- Metricas de Evaluacion ---")
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

# SUPONIENDO QUE EL CÓDIGO NO SE CORTA POR LOS NAN: 

## La diferencia debería ser mayor. La red neuronal artificial realiza mejores predicciones para la clasificación de los datos. 

La red neuronal aplica Epochs con diferentes datos.
Cada epoch evalua las métricas y va guardando esos datos con los callbacks, a la vez que para el proceso y las métricas no cambian entre epochs. 


La Regresión, en este caso Logística, realiza la predicción en base a la división simple de los datos, pero sin evaluar las métricas en cada subconjunto como en epochs. 

In [ ]:
Si se requiere un aprendizaje y predicción mejores o con más datos, y quiere mejores resultados: Usar RNA o ANN